## Import Statements

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import zipfile
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## Load DataFrame


In [ ]:
test = pd.read_csv('../input/histopathologic-cancer-detection/sample_submission.csv')

print('Test Set Size:', test.shape)

In [ ]:
test.head()

## Extract Test Images

In [ ]:
test_path = "../input/histopathologic-cancer-detection/test/"
print('Test Images:', len(os.listdir(test_path)))

In [ ]:
def append_ext(fn):
    return fn+".tif"

In [ ]:
test['id'] = test['id'].apply(append_ext)
test.head()

## Data Generator

In [ ]:
BATCH_SIZE = 64

test_datagen = ImageDataGenerator(rescale=1/255)

test_loader_1 = test_datagen.flow_from_dataframe(
    dataframe = test,
    directory = test_path,
    x_col = 'id',
    batch_size = BATCH_SIZE,
    shuffle = False,
    class_mode = None,
    target_size = (32,32)
)

test_loader_2 = test_datagen.flow_from_dataframe(
    dataframe = test,
    directory = test_path,
    x_col = 'id',
    batch_size = BATCH_SIZE,
    shuffle = False,
    class_mode = None,
    target_size = (64,64)
)

## Load Model

In [ ]:
model_1 = keras.models.load_model('../input/cancer-detection-models/cancer_detection_model_v16.h5')

model_1.summary()

model_2 = keras.models.load_model('../input/cancer-detection-models/cancer_detection_model_v04.h5')

model_3 = keras.models.load_model('../input/cancer-detection-models/cancer_detection_model_v01.h5')


## Test Predictions

In [ ]:
test_probs_1 = model_1.predict(test_loader_2)
test_probs_2 = model_2.predict(test_loader_1)
test_probs_3 = model_3.predict(test_loader_1)
print(test_probs_1.shape)

final_probs = (test_probs_1[:,1] + test_probs_2[:,1] + test_probs_3[:,1])/3

## Prepare Submission

In [ ]:
submission = pd.read_csv('../input/histopathologic-cancer-detection/sample_submission.csv')
submission.head()

In [ ]:
submission.label = final_probs
submission.head()

In [ ]:
submission.to_csv('submission.csv', header=True, index=False)